##  Web Scraping: Daily Mail

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time

### Obtain list of news from the coverpage

URL definition: 

In [2]:
# url definition
url = "https://www.dailymail.co.uk"

List of news:

In [3]:
# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
len(coverpage_news)

97

Now we have a list in which every element is a news article:

In [4]:
coverpage_news[4]

<h2 class="linkro-darkred">
    <a href="/news/article-7833031/BBC-accused-biased-climate-change-Brexit.html" itemprop="url">BBC is accused of being biased on climate change and Brexit... by its own Radio 4 guest editor Charles Moore </a>
  </h2>

### Let's extract the text from the articles:

First, we'll define the number of articles we want: 

In [5]:
number_of_articles = 5

In [6]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
        
    # Getting the link of the article
    link = url + coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('p', class_='mol-para-with-font')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(body)):
        paragraph = body[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    # Removing special characters
    final_article = re.sub("\\xa0", "", final_article)
        
    news_contents.append(final_article)

Let's put them into:

- a dataset which will the input of the models (df_features)
- a dataset with the title and the link (df_show_info)

In [7]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [8]:
df_features

,Article Content
0,Prince Harry and Meghan Markle spent their fir...
1,"A boy, 16, was repeatedly stabbed in front of ..."
2,Britons will toast in 2020 basking in the warm...
3,A resident in Salisbury was the first person i...
4,The BBC has been accused of being biased on to...


In [10]:
df_show_info

,Article Title,Article Link
0,Meghan and Harry's £11m Christmas hideaway in ...,https://www.dailymail.co.uk/news/article-78325...
1,"Knife attacker slashes boy, 16, repeatedly in ...",https://www.dailymail.co.uk/news/article-78328...
2,Britain is set for warmest New Year's Eve in 1...,https://www.dailymail.co.uk/news/article-78323...
3,Britain's first unwanted present was returned ...,https://www.dailymail.co.uk/news/article-78327...
4,BBC is accused of being biased on climate chan...,https://www.dailymail.co.uk/news/article-78330...


### Time Elapsed

We are interested in how much time the script takes to get the news because this will impact directly on user experience. For this, we'll put it all into a single function and then call it:

In [11]:
def get_news_dailymail():
    
    # url definition
    url = "https://www.dailymail.co.uk"
    
    # Request
    r1 = requests.get(url)
    r1.status_code

    # We'll save in coverpage the cover page content
    coverpage = r1.content

    # Soup creation
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    # News identification
    coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
    len(coverpage_news)
    
    number_of_articles = 5

    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []

    for n in np.arange(0, number_of_articles):

        # Getting the link of the article
        link = url + coverpage_news[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('p', class_='mol-para-with-font')

        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(body)):
            paragraph = body[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

         # Removing special characters
        final_article = re.sub("\\xa0", "", final_article)
        
        news_contents.append(final_article)
        

    # df_features
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })

    # df_show_info
    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'Daily Mail'})
    
    return (df_features, df_show_info)

In [12]:
start = time.time()
x, y = get_news_dailymail()
end =time.time()
te = end-start
print("The time elapsed is %f seconds" %(te))

The time elapsed is 29.058333 seconds


Really slow. We won't include in the app.